In [1]:
import pandas as pd

In [4]:
df_train = pd.read_csv("DF_modelo.csv", index_col=0)
df_train

,ID,Inicio Temporada,EDAD,FEC_ALTA,% de asistencia,CUOTA_PAGADA,NOMBRE_SECTOR,Renuevan / No Renuevan
0,7b259c2e043143b,2015,85,9,20.689655,748.00,14,0
1,0f8a86a92fc222d,2015,32,0,44.827586,935.00,14,1
2,3dabbd4e5974ea9,2015,83,0,65.517241,467.00,0,1
3,df72af73ff33004,2015,54,0,68.965517,519.00,0,1
4,e660f665850d2b5,2015,61,0,72.413793,408.00,9,1
...,...,...,...,...,...,...,...,...
155287,036ef2caef5e700,2021,55,11,94.117647,715.62,14,1
155288,73738fb86ca255a,2021,30,14,82.352941,631.43,14,1
155289,2e21500212c12bf,2021,79,14,82.352941,568.29,14,1
155290,caf066457e10021,2021,86,9,94.117647,568.29,14,1


In [7]:
df_train.drop(columns=["ID"], inplace=True)

In [26]:
df_train[df_train["Inicio Temporada"]<2020]

,Inicio Temporada,EDAD,FEC_ALTA,% de asistencia,CUOTA_PAGADA,NOMBRE_SECTOR,Renuevan / No Renuevan
0,2015,85,9,20.689655,748.0,14,0
1,2015,32,0,44.827586,935.0,14,1
2,2015,83,0,65.517241,467.0,0,1
3,2015,54,0,68.965517,519.0,0,1
4,2015,61,0,72.413793,408.0,9,1
...,...,...,...,...,...,...,...
133161,2019,55,11,100.000000,1009.0,14,1
133162,2019,30,14,94.117647,1121.0,14,1
133163,2019,56,9,94.117647,831.7,14,0
133164,2019,86,9,94.117647,1009.0,14,1


In [81]:
x_train = df_train[df_train["Inicio Temporada"]<2020].drop(columns=["Renuevan / No Renuevan", "Inicio Temporada"])
y_train = df_train[df_train["Inicio Temporada"]<2020].iloc[:,-1]
x_test = df_train[df_train["Inicio Temporada"]==2021].drop(columns=["Renuevan / No Renuevan", "Inicio Temporada"])
y_test = df_train[df_train["Inicio Temporada"]==2021].iloc[:,-1]

In [82]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=0)

In [83]:
clf.feature_importances_

array([0.29620417, 0.13379288, 0.16678434, 0.29002581, 0.11319279])

In [30]:
#df_test = pd.read_csv("DF_modelo_test.csv", index_col=0).drop(columns=["ID"])

In [84]:
predictions = clf.predict(x_test)

In [85]:
sum(predictions)/len(predictions)

0.7282579123968441

In [86]:
y_test

133166    1
133167    1
133168    1
133169    1
133170    1
         ..
155287    1
155288    1
155289    1
155290    1
155291    1
Name: Renuevan / No Renuevan, Length: 22054, dtype: int64

RANDOM FOREST CLASSIFIER

In [94]:
from sklearn.ensemble import RandomForestClassifier

In [99]:
clf = RandomForestClassifier(max_depth=5, max_features=20, random_state=0)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=5, max_features=5, random_state=0)

In [101]:
predictions = clf.predict(x_test)

In [102]:
sum(predictions)/len(predictions)

0.9610501496327197

In [107]:
clf.score(y_test.values.reshape(-1, 1), predictions)

c:\Users\jamf1\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but RandomForestClassifier is expecting 5 features as input.